In [1]:
import pandas as pd
import seaborn as sns
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import csv
from IPython.core.display import display, HTML
import re

In [2]:
def load_dealroom():
    # Carrega o dataframe
    dealroom_path = "C:\\Users\\alex\\PycharmProjects\\chamada\\data\\processado\\dealroom\\companies_11000.xlsx"
    dfd = pd.read_excel(dealroom_path)
    dfd.drop_duplicates(subset=["id"], inplace=True)
    # Seleciona as empresas com os seguintes tipos de investimento recebidos
    select = dfd['LAST ROUND'].isin(['ANGEL', 'SEED', 'EARLY VC', 'SERIES A', 'SERIES B', 'SERIES C', 'SERIES D', 'LATE VC', 'ACQUISITION'])
    dfd = dfd[select]
    # Transforma o atributo LAST ROUND em categórico
    cat_type = CategoricalDtype(categories=['ANGEL', 'SEED', 'EARLY VC', 'SERIES A', 'SERIES B', 'SERIES C', 'SERIES D', 'LATE VC', 'ACQUISITION'], ordered=True)
    dfd['LAST ROUND'] = dfd['LAST ROUND'].astype(cat_type)
    dfd['EMPLOYEES'] = dfd['EMPLOYEES'].str[1:]
    cat_employees = CategoricalDtype(categories=['11-50', '51-200'], ordered=True)
    dfd['EMPLOYEES'] = dfd['EMPLOYEES'].astype(cat_employees)
    # Conta a quantidade de fundadores homens
    dfd['N_MALES'] = dfd['FOUNDERS GENDERS'].str.count(r"\b(male)\b")
    # Conta a quantidade de fundadores mulheres
    dfd['N_FEMALES'] = dfd['FOUNDERS GENDERS'].str.count(r"\b(female)\b")
    # Conta a quantidade de fundadores seriais
    dfd['N_SERIAL_FOUNDER'] = dfd['FOUNDERS IS SERIAL'].str.count(r"\b(yes)\b")
    dfd['N_NOT_SERIAL_FOUNDER'] = dfd['FOUNDERS IS SERIAL'].str.count(r"\b(no)\b")
    dfd['SERIAL_FOUNDER_PROPORTION'] = (100*dfd['N_SERIAL_FOUNDER'] / (dfd['N_SERIAL_FOUNDER'] + dfd['N_NOT_SERIAL_FOUNDER'])).round()
    dfd['GENDER_PROPORTION'] = (100 * dfd['N_MALES'] / (dfd['N_MALES'] + dfd['N_FEMALES'])).round()
    # Transforma a string em objeto datetime
    dfd['launch_date'] = dfd['LAUNCH DATE'].apply(lambda x: datetime.strptime(x, "%Y; %B") if len(str(x)) > 4 else datetime.strptime(f"{x}, July", "%Y, %B") )
    # Transforma a data em ano
    dfd['launch_year'] = dfd['LAUNCH DATE'].apply(lambda x: int(re.findall("\d{4}", str(x))[0]) )
    # Transforma a string em objeto datetime
    dfd['last_funding_date'] = dfd['LAST FUNDING DATE'].apply(lambda x: datetime.strptime(str(x), "%b/%Y") if len(str(x)) > 4 else datetime.strptime(f"jul/{x}", "%b/%Y"))
    # Transforma a data em ano
    dfd['last_funding_year'] = dfd['LAST FUNDING DATE'].apply(lambda x: int(re.findall("\d{4}",str(x))[0]) )
    dfd['last_funding_period_years'] = 2020 - dfd['launch_year']
    dfd['last_funding_period_days'] = ((datetime.today() - dfd['launch_date']) / np.timedelta64(1, 'D')).round()
    # Calcula a quantidade de dias desde o lançamento até a data da ultima rodada de investimento
    dfd['last_funding_period_days2'] = ((dfd['last_funding_date'] - dfd['launch_date']) / np.timedelta64(1, 'D')).round()
    dfd['average_funding_period'] = dfd['last_funding_period_days'] / dfd['TOTAL ROUNDS NUMBER']
    dfd['average_funding_period2'] = dfd['last_funding_period_days2'] / dfd['TOTAL ROUNDS NUMBER']

    # Filtra empresas que já tiveram pelo menos 3 rodadas de investimento, ou 
    #   empresas que já estão há pelo menos 5 anos no mercado.
    # dfd = dfd[(dfd['TOTAL ROUNDS NUMBER'] >= 3) | (dfd['last_funding_period_days2'] >= 5*365)]
    return dfd

In [6]:
def load_crunchbase(year_limit=None):
    ## Carrega o arquivo csv consolidado
    # path_startup_consol = "/home/alex/vscode/data/original/crunchbase/consolidado/startups.csv"
    path_startup_consol = "C:\\Users\\alex\\PycharmProjects\\chamada\\data\\original\\crunchbase\\consolidado\\startups.csv"
    df = pd.read_csv(path_startup_consol)
    df.drop(261, inplace=True) ## Tinha um valor errado na data
    df.rename(columns={"Unnamed: 0": "id"}, inplace=True)
    ## 3) Faz um filtro por empresas antigas (anteriores a 2017)
    df['Founded Date'] = df['Founded Date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d") if type(x) == str else x)
    df['Founded_Year'] = df['Founded Date'].apply(lambda x: x.year)
    df['Last Funding Date'] = df['Last Funding Date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d") if type(x) == str else x)
    # 4) Calcula o tempo médio das rodas de investimento
    df['inter_funding_period'] = datetime.today() - df['Founded Date']
    df['inter_funding_days'] = df['inter_funding_period'].apply(
                                        lambda x: x/np.timedelta64(1, 'D'))
    df['average_if_period'] = df['inter_funding_days'] / df['Number of Funding Rounds']
    df['inter_funding_period2'] = df['Last Funding Date'] - df['Founded Date']
    df['inter_funding_days2'] = df['inter_funding_period'].apply(
                                        lambda x: x/np.timedelta64(1, 'D'))
    df['average_if_period2'] = df['inter_funding_days'] / df['Number of Funding Rounds']
    if year_limit:
        df = df.loc[df['Founded_Year'] <= year_limit, :]
    return df